In [4]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import pandas as pd
import os
# from train_cve_model import fine_tune_cve_model


/Volumes/Kingsman/AEN/anaconda3/envs/py313/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import pandas as pd
import numpy as np
df= pd.read_csv('redhat_advisories/output/redhat_packages.csv')

In [6]:
df.head(2)

,advisory_id,advisory_title,severity,summary,details,topic,product_id,product_name,full_product_name,category,vulnerability_count,cve_list,cve_titles,cve_descriptions,file_path
0,RHSA-2013:0580,Red Hat Security Advisory: cups security update,Moderate,Updated cups packages that fix one security is...,The Common UNIX Printing System (CUPS) provide...,NaN,5Server-5.9.Z,Red Hat Enterprise Linux (v. 5 server),Red Hat/Red Hat Enterprise Linux/Red Hat Enter...,product_name,1,CVE-2012-5519,CVE-2012-5519: cups: privilege escalation for ...,"CVE-2012-5519: CUPS 1.4.4, when running in cer...",redhat_advisories_extracted/2013/rhsa-2013_058...
1,RHSA-2013:0580,Red Hat Security Advisory: cups security update,Moderate,Updated cups packages that fix one security is...,The Common UNIX Printing System (CUPS) provide...,NaN,6Client-6.4.z,Red Hat Enterprise Linux Desktop (v. 6),Red Hat/Red Hat Enterprise Linux/Red Hat Enter...,product_name,1,CVE-2012-5519,CVE-2012-5519: cups: privilege escalation for ...,"CVE-2012-5519: CUPS 1.4.4, when running in cer...",redhat_advisories_extracted/2013/rhsa-2013_058...


In [7]:


def strip_cve_id(desc):
    if pd.isna(desc):
        return None
    return ' '.join(word for word in str(desc).split() if not word.upper().startswith("CVE-"))

df["cve_descriptions"] = df["cve_descriptions"].apply(strip_cve_id)

In [12]:
def safe_text(val):
    return str(val) if pd.notna(val) else ""

train_examples = [
    InputExample(texts=[
        safe_text(row["product_id"]),
        safe_text(row["cve_descriptions"]),
        safe_text(row["details"]),
        safe_text(row["summary"])
    ])
    for _, row in df.iterrows()
]

In [14]:
df.head(2)

,advisory_id,advisory_title,severity,summary,details,topic,product_id,product_name,full_product_name,category,vulnerability_count,cve_list,cve_titles,cve_descriptions,file_path
0,RHSA-2013:0580,Red Hat Security Advisory: cups security update,Moderate,Updated cups packages that fix one security is...,The Common UNIX Printing System (CUPS) provide...,NaN,5Server-5.9.Z,Red Hat Enterprise Linux (v. 5 server),Red Hat/Red Hat Enterprise Linux/Red Hat Enter...,product_name,1,CVE-2012-5519,CVE-2012-5519: cups: privilege escalation for ...,"CUPS 1.4.4, when running in certain Linux dist...",redhat_advisories_extracted/2013/rhsa-2013_058...
1,RHSA-2013:0580,Red Hat Security Advisory: cups security update,Moderate,Updated cups packages that fix one security is...,The Common UNIX Printing System (CUPS) provide...,NaN,6Client-6.4.z,Red Hat Enterprise Linux Desktop (v. 6),Red Hat/Red Hat Enterprise Linux/Red Hat Enter...,product_name,1,CVE-2012-5519,CVE-2012-5519: cups: privilege escalation for ...,"CUPS 1.4.4, when running in certain Linux dist...",redhat_advisories_extracted/2013/rhsa-2013_058...


In [26]:
# Load base model
model_name="basel/ATTACK-BERT"  # or "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)


# train_examples = [
#         InputExample(texts=[df.product_id, df.cve_descriptions,df.details,df.summary])
#         for _, row in df.iterrows()
#     ]
# # Prepare DataLoader and loss
batch_size=5
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model)

In [27]:
# Train
epochs=3
warmup_steps=100
show_progress_bar=True

# Run fine-tuning
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    show_progress_bar=show_progress_bar
    )


Step,Training Loss


RuntimeError: MPS backend out of memory (MPS allocated: 11.76 GB, other allocations: 6.39 GB, max allowed: 18.13 GB). Tried to allocate 5.62 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
#Save

model.save("./rhel_cve_model1")